# Full notebook
This notebook has the whole process put together:
1. Loading dataset
2. Defining grammar
3. Searching through the grammar and evaluating candidate pipelines

## 0. Imports

In [1]:
# # uncomment the following if not all packages are added.

# import Pkg
# using Pkg
# Pkg.add("HTTP")
# Pkg.add("JSON")
# Pkg.add("DataFrames")
# Pkg.add("OpenML")
# Pkg.add("DataFrames") 
# Pkg.add("CSV") 
# Pkg.add("Suppressor")
# Pkg.add("StatsBase")
# Pkg.add("ScikitLearn")

In [3]:
using ScikitLearn
using ScikitLearn.Pipelines: Pipeline, FeatureUnion
using ScikitLearn.CrossValidation: cross_val_score
using XGBoost
using Revise
using Random
using Statistics: mean
using ExprRules: get_executable
using Suppressor
using Random
using Dates

include("./lib/Herb.jl/src/Herb.jl")
include("./lib/HerbGrammar.jl/src/HerbGrammar.jl")
include("./lib/HerbData.jl/src/HerbData.jl")
include("./lib/HerbEvaluation.jl/src/HerbEvaluation.jl")
include("./lib/HerbConstraints.jl/src/HerbConstraints.jl")
include("./lib/HerbSearch.jl/src/HerbSearch.jl")
include("helper.jl")

get_class_type_dataset (generic function with 1 method)

In [4]:
# import the sk-learn functions
@sk_import decomposition: (PCA)
@sk_import preprocessing: (StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, Binarizer, PolynomialFeatures)
@sk_import feature_selection: (VarianceThreshold, SelectKBest, SelectPercentile, SelectFwe, RFE)
@sk_import tree: (DecisionTreeClassifier)
@sk_import ensemble: (RandomForestClassifier, GradientBoostingClassifier)
@sk_import linear_model: (LogisticRegression)
@sk_import neighbors: (NearestNeighbors)
@sk_import svm: (LinearSVC)

PyObject <class 'sklearn.svm._classes.LinearSVC'>

## 1. Loading datasets

In [5]:
# Datasets: seeds (1499), diabetes (37), tic-tac-toe (50), steel-plates-fault (1504), 

# load dataset
# dataset = get_dataset(61)

# it does not work for the seeds table dataset!
dataset = get_dataset(1499)

# does not work either on dataset 1464!

# shuffle the dataset
dataset_shuffled = dataset[shuffle(1:end), :]

# split into train and test sets (90:10)
split_index = floor(Int, size(dataset_shuffled, 1) * 0.90)
train_df = dataset_shuffled[1:split_index, :]
test_df = dataset_shuffled[split_index+1:end, :]

# show first five entries
first(dataset_shuffled, 5)

Row,V1,V2,V3,V4,V5,V6,V7,Class
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…
1,17.99,15.86,0.8992,5.89,3.694,2.068,5.837,2
2,15.88,14.9,0.8988,5.618,3.507,0.7651,5.091,1
3,10.8,12.57,0.859,4.981,2.821,4.773,5.063,3
4,15.26,14.85,0.8696,5.714,3.242,4.543,5.314,1
5,16.63,15.46,0.8747,6.053,3.465,2.04,5.877,1


In [6]:
# show metadata
print("size: ", size(dataset_shuffled))
describe(dataset_shuffled)

size: (210, 8)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,V1,14.8475,10.59,14.355,21.18,0,Float64
2,V2,14.5593,12.41,14.32,17.25,0,Float64
3,V3,0.870999,0.8081,0.87345,0.9183,0,Float64
4,V4,5.62853,4.899,5.5235,6.675,0,Float64
5,V5,3.2586,2.63,3.237,4.033,0,Float64
6,V6,3.7002,0.7651,3.599,8.456,0,Float64
7,V7,5.40807,4.519,5.223,6.55,0,Float64
8,Class,,1,,3,0,"CategoricalValue{String, UInt32}"


In [7]:
# split into features and labels
train_X = train_df[:, 1:end-1]
train_Y = train_df[:, end]
test_X = test_df[:, 1:end-1]
test_Y = test_df[:, end]

# print ratio train/test
ratio = trunc(Int, 10.0 * (size(train_df)[1] / (size(train_df)[1] + size(test_df)[1])))
print("train:test ratio = ", ratio , ":", (10-ratio))

train:test ratio = 9:1

## 2. Defining grammar

In [8]:
grammar = Herb.HerbGrammar.@cfgrammar begin

    # this is the version with multiple classifiers possible
    # START   = CLASSIF | sequence(PRE, CLASSIF)
    # PRE     = PREPROC | FSELECT | sequence(PRE, PRE) | parallel(BRANCH, BRANCH)
    # BRANCH  = PRE | CLASSIF | sequence(PRE, CLASSIF) 

    # this is the version with only one classifier
    START   = Pipeline([CLASSIF]) | Pipeline([PRE, CLASSIF])
    PRE     = PREPROC | FSELECT | ("seq", Pipeline([PRE, PRE]))  | ("par", FeatureUnion([PRE, PRE])) 

    # preprocessing functions
    PREPROC =   
        ("StandardScaler" * string(rand(Int)), StandardScaler()) |
        ("RobustScaler", RobustScaler()) |
        ("MinMaxScaler", MinMaxScaler()) |
        ("MaxAbsScaler", MaxAbsScaler()) |
        ("PCA", PCA()) |
        ("Binarizer", Binarizer()) |
        ("PolynomialFeatures", PolynomialFeatures())

    # feature selection functions
    FSELECT =  
        ("VarianceThreshold", VarianceThreshold()) |
        # ("SelectKBest",  SelectKBest()) |
        ("SelectPercentile",  SelectPercentile()) |
        ("SelectFwe",  SelectFwe()) |
        ("Recursive Feature Elimination",  RFE(LinearSVC())) 

    # classifiers
    CLASSIF =
        ("DecisionTree", DecisionTreeClassifier()) |
        ("RandomForest", RandomForestClassifier()) |
        ("Gradient Boosting Classifier", GradientBoostingClassifier()) |
        ("LogisticRegression", LogisticRegression()) |
        ("NearestNeighborClassifier", NearestNeighbors())
end

1: START = Pipeline([CLASSIF])
2: START = Pipeline([PRE, CLASSIF])
3: PRE = PREPROC
4: PRE = FSELECT
5: PRE = ("seq", Pipeline([PRE, PRE]))
6: PRE = ("par", FeatureUnion([PRE, PRE]))
7: PREPROC = ("StandardScaler" * string(rand(Int)), StandardScaler())
8: PREPROC = ("RobustScaler", RobustScaler())
9: PREPROC = ("MinMaxScaler", MinMaxScaler())
10: PREPROC = ("MaxAbsScaler", MaxAbsScaler())
11: PREPROC = ("PCA", PCA())
12: PREPROC = ("Binarizer", Binarizer())
13: PREPROC = ("PolynomialFeatures", PolynomialFeatures())
14: FSELECT = ("VarianceThreshold", VarianceThreshold())
15: FSELECT = ("SelectPercentile", SelectPercentile())
16: FSELECT = ("SelectFwe", SelectFwe())
17: FSELECT = ("Recursive Feature Elimination", RFE(LinearSVC()))
18: CLASSIF = ("DecisionTree", DecisionTreeClassifier())
19: CLASSIF = ("RandomForest", RandomForestClassifier())
20: CLASSIF = ("Gradient Boosting Classifier", GradientBoostingClassifier())
21: CLASSIF = ("LogisticRegression", LogisticRegression())
22: CLASSI

In [9]:
function insert_name_indexes(p)
    p_start = ""
    i = 1
    while i <= 100
        try
            p = replace(p, """",""" => string(i)*"""",""", count=1)
            p_split = split(p, string(i) * """", """)
            p_start *= p_split[1] * string(i) * """", """
            p = p_split[2]
            i += 1
        catch
            break
        end
    end
    return split(p_start, string(i))[1]
end

insert_name_indexes (generic function with 1 method)

In [10]:
function get_random_pipeline(grammar, max_depth, start_symbol)
    # all pipelines that can be assembled in max_depth steps
    cfe = Herb.HerbSearch.ContextFreeEnumerator(grammar, max_depth, :START)
    cfe_size = deepcopy(cfe)
    
    # find size
    size = 0
    for pipeline in cfe_size
        size += 1
    end

    # Find start program
    ret_pipeline = nothing
    c = 0
    i = abs(rand(Int) % size)
    for pipeline in cfe
        if c == i
            ret_pipeline = pipeline
            break
        end
        c = c + 1
    end
    return ret_pipeline
end

get_random_pipeline (generic function with 1 method)

## 3. Search

In [11]:
"""
Fits the pipeline to the training set and measures the accuracy on test set.
input:  pipeline, train_X, train_Y, test_X, test_Y
output: accuracy of pipeline
"""
function evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)

    # # this gives the following warning often, so it is suppressed for now.
    # # ConvergenceWarning: lbfgs failed to converge
    @suppress_err begin
        try
            # fit the pipeline
            # print(pipeline)
            # print(" - ")
            model = ScikitLearn.fit!(pipeline, Matrix(train_X), Array(train_Y))

            # make predictions
            predictions = ScikitLearn.predict(model, Matrix(test_X))

            # measure the accuracy
            accuracy = mean(predictions .== test_Y)
            return accuracy
        catch e
            # println("Caught error [in evaluate_pipeline()]: ", e)
            return 0.0
        end
    end
end

evaluate_pipeline

In [12]:
"""Trains the pipeline and returns 1-accuracy"""
function pipeline_cost_function(pipeline, train_X, train_Y, test_X, test_Y)
    return 1.0 - evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)
end

pipeline_cost_function

In [13]:
"""This function enumerates the grammar and finds the best pipeline. """
function find_best_pipeline_with_bfs_search(grammar, train_X, train_Y, test_X, test_Y, search_depth)
    best_accuracy = -1.0
    best_pipeline = nothing

    # enumerate the gramamar
    enumerator = Herb.HerbSearch.ContextFreeEnumerator(grammar, search_depth, :START)
    for rule in enumerator
        try
            # get pipeline and calculate accuracy
            pipeline = eval(Herb.HerbSearch.rulenode2expr(rule, grammar))
            accuracy = evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)

            # update best pipeline
            if (accuracy > best_accuracy) 
                best_accuracy = accuracy
                best_pipeline = pipeline
            end
            
            # print accuracy of pipeline
            print("\n accuracy: ", round(accuracy, digits=2), " by ", string(pipeline))
        catch
            continue
        end
    end
    return (best_accuracy, best_pipeline)
end

find_best_pipeline_with_bfs_search

In [14]:
# find the best pipeline and accuracy with depth 2
(best_accuracy, best_pipeline) = find_best_pipeline_with_bfs_search(grammar, train_X, train_Y, test_X, test_Y, 2)

# print result
println("\n\nBest pipeline: ", round(best_accuracy, digits=2))
print(best_pipeline)


 accuracy: 0.76 by 

Pipeline(Tuple{Any, Any}[("DecisionTree", PyObject DecisionTreeClassifier())], Any[PyObject DecisionTreeClassifier()])
 accuracy: 0.81 by Pipeline(Tuple{Any, Any}[("RandomForest", PyObject RandomForestClassifier())], Any[PyObject RandomForestClassifier()])


 accuracy: 0.76 by Pipeline(Tuple{Any, Any}[("Gradient Boosting Classifier", PyObject GradientBoostingClassifier())], Any[PyObject GradientBoostingClassifier()])
 accuracy: 0.81 by Pipeline(Tuple{Any, Any}[("LogisticRegression", PyObject LogisticRegression())], Any[PyObject LogisticRegression()])
 accuracy: 0.0 by Pipeline(Tuple{Any, Any}[("NearestNeighborClassifier", PyObject NearestNeighbors())], Any[PyObject NearestNeighbors()])

Best pipeline: 0.81


Pipeline(

Tuple{Any, Any}[

("RandomForest", PyObject RandomForestClassifier())]

, Any[PyObject RandomForestClassifier()])

In [15]:
test_pipeline = Pipeline([("DecisionTree", DecisionTreeClassifier())])
pipeline_cost_function(test_pipeline, train_X, train_Y, test_X, test_Y)

0.23809523809523814

In [24]:
using Random
abstract type ExpressionIterator end


# TreeNode definition.
mutable struct TreeNode
    state
    visits::Int
    wins::Float64
    depth::Int
    children::Array{TreeNode, 1}
    parent::Union{TreeNode, Nothing}
end

# Create a new TreeNode.
function TreeNode(state, depth, parent)
    return TreeNode(state, 0, 0.0, depth, TreeNode[], parent)
end

# Select the child node with the highest UCT score.
function select_child(node::TreeNode, c)
    total_visits = sum(n.visits for n in values(node.children))
    best_score = -Inf
    best_child = nothing

    for child in node.children
        if child.visits == 0
            score = Inf  # Set a high score for unvisited nodes
        else
            exploration_term = c * sqrt(log(total_visits) / child.visits)
            score = child.wins / child.visits + exploration_term
        end

        if score > best_score
            best_score = score
            best_child = child
        end
    end

    return best_child
end

# Expands a chosen node by adding all configurations as child nodes an picking one at random.
function expand_node(node, grammar)
    # Checks in memory if the rules have already been enumerated once to save time.
    if mem[node.depth+1] == nothing
        enumerator = Herb.HerbSearch.ContextFreeEnumerator(grammar, node.depth+1, :START)
        rules = []
        for rule in enumerator
            push!(rules, rule)
        end
        mem[node.depth+1] = rules
        push!(mem, nothing)
    else
        rules = mem[node.depth+1]
    end
    
    # Checks what the configuration of the current node is and filters out all configurations that do not match.
    temp = [string(node.state.children[i])[1:end-1] for i in eachindex(node.state.children)]
    filtered_rules = filter(x -> all(contains(string(x), s) for s in temp), rules)[2:end]
    
    # Create an array of child nodes and assign them to the current node.
    child_nodes = TreeNode[]
    for child in filtered_rules
        push!(child_nodes , TreeNode(child, node.depth+1, node))
    end
    node.children = child_nodes
    
    return rand(node.children)
end

# The simulation step tries to evaluate a pipeline and returns its accuracy.
function simulate(state)
    accuracy = 0
    try
        #pipeline = eval(Herb.HerbSearch.rulenode2expr(state, grammar))
        pipeline = eval(Meta.parse(insert_name_indexes(string(Herb.HerbSearch.rulenode2expr(state, grammar)))))
        accuracy = evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)
        #println("Accuracy: ", accuracy)
    catch
        accuracy = 0
    end
    return accuracy
end

# The backpropagation step updates the visits and wins fields of the given node and its parent node.
function backpropagate(node::TreeNode, result)
    while node != nothing
        node.visits += 1
        node.wins += result
        node = node.parent
    end
end

# Perform Monte Carlo Tree Search
function mcts(root_state, max_iterations, grammar, c)
    best_pipeline_score = 0 
    best_pipeline_conf = nothing
    for _ in 1:max_iterations
        node = root_node
        # Selection 
        while !isempty(node.children) 
            node = select_child(node, c)
        end
        # Expansion
        if node.visits > 0
            node = expand_node(node, grammar)
        end

        # Simulation
        reward = simulate(node.state)
        if reward > best_pipeline_score
            best_pipeline_score = reward
            best_pipeline_conf = node.state
        end

        # Backpropagation
        backpropagate(node, reward)
    end
    return best_pipeline_conf, best_pipeline_score
end

# Creates an inital state to use as a starting point for the algorithm. 
function select_initial_state(grammar, max_depth, start_symbol)
    initial_state = TreeNode(:EMPTY, 1, nothing)  # Create a root node with an empty symbol
    # Expand the root node by adding the first layer of options
    
    child_nodes = TreeNode[]
    enumerator = Herb.HerbSearch.ContextFreeEnumerator(grammar, max_depth, start_symbol)
    rules = []
    for expression in enumerator
        child = TreeNode(expression, 2, initial_state)
        push!(rules, expression)
        push!(child_nodes, child)
    end

    push!(mem, rules)
    push!(mem, nothing)

    initial_state.children = child_nodes
    
    return initial_state
end

# Initialize the memory array for the memoization of certain configuration.
mem = []
push!(mem, nothing)
# Generate the root node.
root_node = select_initial_state(grammar, 2, :START)

# Perform Monte Carlo Tree Search!
best_pipeline = mcts(root_node, 1000, grammar, 1.42)


(2{5{3{11}3{9}}21}, 1.0)

In [17]:
function expand_node(node, grammar)
    if mem[node.depth+1] == nothing
        enumerator = Herb.HerbSearch.ContextFreeEnumerator(grammar, node.depth+1, :START)
        rules = []
        for rule in enumerator
            push!(rules, rule)
        end
        mem[node.depth+1] = rules
        push!(mem, nothing)
    else
        rules = mem[node.depth+1]
    end
    
    temp = [string(node.state.children[i])[1:end-1] for i in eachindex(node.state.children)]
    #println(temp)
    filtered_rules = filter(x -> all(contains(string(x), s) for s in temp), rules)[2:end]
    #println(filtered_rules)
    child_nodes = TreeNode[]
    for child in filtered_rules
        push!(child_nodes , TreeNode(child, node.depth+1, node))
    end
    #println(child_nodes)
    node.children = child_nodes
    
    return rand(node.children)
end

#test_node = TreeNode(depth2, 2, nothing)

#2{3{11}19}
#depth2 = best_pipeline[1]
#depth3 = node
#depth4 = node
#depth5 = node
#node = expand_node(depth5, grammar)




expand_node (generic function with 1 method)